In [1]:
import pandas as pd
import json

# Rutas a los archivos
path_users = '../data/raw/users.csv'
path_viewing = '../data/raw/viewing_sessions.csv'
path_content = '../data/raw/content.json'

# Cargar los datos de los archivos CSV en DataFrames
df_users = pd.read_csv(path_users)
df_viewing = pd.read_csv(path_viewing)

# Cargar el archivo JSON
with open(path_content, 'r') as f:
    data_content = json.load(f)

# Convertir el contenido del JSON en DataFrames
movies_list = data_content['movies']
series_list = data_content['series']
df_movies = pd.DataFrame(movies_list)
df_series = pd.DataFrame(series_list)

print("DataFrames cargados exitosamente")

DataFrames cargados exitosamente


In [2]:
print("--- Información de df_users ---")
df_users.info()
print("\n--- Información de df_viewing ---")
df_viewing.info()
print("\n--- Información de df_movies ---")
df_movies.info()
print("\n--- Información de df_series ---")
df_series.info()

--- Información de df_users ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   user_id                 5000 non-null   object 
 1   age                     5000 non-null   int64  
 2   country                 5000 non-null   object 
 3   subscription_type       5000 non-null   object 
 4   registration_date       5000 non-null   object 
 5   total_watch_time_hours  5000 non-null   float64
dtypes: float64(1), int64(1), object(4)
memory usage: 234.5+ KB

--- Información de df_viewing ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222785 entries, 0 to 222784
Data columns (total 8 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   session_id              222785 non-null  object 
 1   user_id                 222785 non-null  object 
 2   content_

In [3]:
print("--- Primeras 5 filas de df_users ---")
print(df_users.head())
print("\n--- Primeras 5 filas de df_viewing ---")
print(df_viewing.head())
print("\n--- Primeras 5 filas de df_movies ---")
print(df_movies.head())
print("\n--- Primeras 5 filas de df_series ---")
print(df_series.head())

--- Primeras 5 filas de df_users ---
  user_id  age    country subscription_type registration_date  \
0   U0001   21  Argentina             Basic        2022-10-02   
1   U0002   44     Mexico          Standard        2022-09-30   
2   U0003   20  Argentina          Standard        2023-01-24   
3   U0004   39       Peru          Standard        2022-12-15   
4   U0005   28   Colombia             Basic        2022-05-19   

   total_watch_time_hours  
0                    58.0  
1                    75.3  
2                   360.0  
3                   513.0  
4                   192.8  

--- Primeras 5 filas de df_viewing ---
  session_id user_id content_id  watch_date  watch_duration_minutes  \
0    S000001   U0001       S001  2024-02-16                       9   
1    S000002   U0001       M110  2024-08-06                      30   
2    S000003   U0001       S005  2024-03-20                      18   
3    S000004   U0001       M105  2024-02-24                      77   
4    S000

## Data Cleaning

In [5]:
# Convert 'registration_date' to datetime in the users DataFrame
df_users['registration_date'] = pd.to_datetime(df_users['registration_date'])

# Convert 'watch_date' to datetime in the viewing_sessions DataFrame
df_viewing['watch_date'] = pd.to_datetime(df_viewing['watch_date'])

print("Date columns successfully converted.")

Date columns successfully converted.


In [6]:
# Merge df_users and df_viewing on the 'user_id' column
df_combined = pd.merge(df_users, df_viewing, on='user_id', how='inner')

print("Combined DataFrame created successfully.")
print("\nFirst 5 rows of the combined DataFrame:")
print(df_combined.head())

print("\nInformation of the new combined DataFrame:")
df_combined.info()

Combined DataFrame created successfully.

First 5 rows of the combined DataFrame:
  user_id  age    country subscription_type registration_date  \
0   U0001   21  Argentina             Basic        2022-10-02   
1   U0001   21  Argentina             Basic        2022-10-02   
2   U0001   21  Argentina             Basic        2022-10-02   
3   U0001   21  Argentina             Basic        2022-10-02   
4   U0001   21  Argentina             Basic        2022-10-02   

   total_watch_time_hours session_id content_id watch_date  \
0                    58.0    S000001       S001 2024-02-16   
1                    58.0    S000002       M110 2024-08-06   
2                    58.0    S000003       S005 2024-03-20   
3                    58.0    S000004       M105 2024-02-24   
4                    58.0    S000005       M148 2024-08-18   

   watch_duration_minutes  completion_percentage device_type quality_level  
0                       9                   18.8     Desktop            4K  


A pd.merge() is essential because it links your data together.

The raw files you have are separate and can't be analyzed together. For example, df_users has a user's age and country, but it doesn't know what they watched. df_viewing knows what content was watched, but it doesn't know the user's age.

pd.merge() solves this by using a shared column—the user_id—to combine the tables. This creates a new, single table that connects every user's personal details to every single one of their viewing sessions.

### 1. Descriptive Metrics: Data Summary

In [8]:
# Combine movie and series DataFrames into a single content DataFrame
df_content = pd.concat([df_movies, df_series], ignore_index=True)

# Merge df_combined (users + sessions) with the new df_content
df_final = pd.merge(df_combined, df_content, on='content_id', how='inner')

# Analyze key metrics of the final DataFrame
print("--- Summary of Key Metrics ---")
print(df_final[['age', 'watch_duration_minutes', 'completion_percentage', 'total_watch_time_hours']].describe())

--- Summary of Key Metrics ---
                 age  watch_duration_minutes  completion_percentage  \
count  222785.000000           222785.000000          222785.000000   
mean       41.538551               78.907871              78.384990   
std        13.966006               45.007097              21.462752   
min        18.000000                2.000000              10.000000   
25%        29.000000               37.000000              66.300000   
50%        42.000000               79.000000              84.500000   
75%        54.000000              115.000000              96.000000   
max        65.000000              180.000000             100.000000   

       total_watch_time_hours  
count           222785.000000  
mean               352.365682  
std                232.188045  
min                  9.800000  
25%                169.500000  
50%                324.100000  
75%                481.200000  
max               1093.100000  


### 2. Analysis by Category


In [9]:
# Group by subscription type and calculate average metrics
df_subscription_summary = df_final.groupby('subscription_type').agg({
    'watch_duration_minutes': 'mean',
    'completion_percentage': 'mean',
    'total_watch_time_hours': 'mean'
}).reset_index()

print("--- Average Metrics by Subscription Type ---")
print(df_subscription_summary)

--- Average Metrics by Subscription Type ---
  subscription_type  watch_duration_minutes  completion_percentage  \
0             Basic               68.585605              68.284554   
1           Premium               88.342635              87.893649   
2          Standard               80.000802              79.217029   

   total_watch_time_hours  
0              258.526403  
1              507.173589  
2              303.058755  
